In [1]:
from IPython.core.display import HTML
display(HTML('<style>.container { width:95% !important; } </style>'))

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ito_diffusions import *
from statsmodels.graphics.tsaplots import plot_acf
plt.style.use('dark_background')

In [ ]:
params = {
    'font.size': 14,
    'figure.figsize': (22, 18),
    'axes.labelsize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
}
plt.rcParams.update(params)

In [ ]:
SEED = 12345
T = int(1e3)
integrated = True

## Autoregressive model AR(p)

$$X_t = \mu + \sum_{i=1}^p a_{i} X_{t-i} + Z_t\,.$$

In [ ]:
fig, axess = plt.subplots(nrows=5, ncols=2)

a_ = [
    [],
    [0.3],
    [0.9],
    [0.3, 0.3],
    [0.9, -0.8],
]

X = AR(x0=[0.0, 0.0], mu=0.0, T=T, vol=1.0)

for i, axes in enumerate(axess):
    a = a_[i]
    X.a = a
    # Rerun with same rng to see the difference between models using the same noise distribution
    X.rng = np.random.default_rng(SEED)
    df = X.simulate()
    
    ax = axes[0]
    if integrated:
        df.cumsum().plot(kind='line', ax=ax)
    else:
        df.plot(kind='line', ax=ax)

    if X.p > 0:
        ax.set_xlabel('time\n$\mu$={:.1f}, a={}'.format(X.mu, X.a))
    else:
        ax.set_xlabel('time\n$\mu$={:.1f}'.format(X.mu))
    ax.set_ylabel('spot')
    ax.legend(loc='upper right')
    ax.set_title('AR({:.0f})'.format(X.p))
    
    ax = axes[1]
    plot_acf(df, ax=ax)
    
plt.tight_layout()
plt.show()

## Moving-average model MA(q)

$$X_t = \mu + Z_t + \sum_{j=1}^q b_{j} Z_{t-j}\,.$$

In [ ]:
fig, axess = plt.subplots(nrows=5, ncols=2)

b_ = [
    [],
    [0.3],
    [0.9],
    [0.3, 0.3],
    [0.9, -0.8],
]

X = MA(x0=[0.0, 0.0], mu=0.0, T=T, vol=1.0)

for i, axes in enumerate(axess):
    b = b_[i]
    X.b = b
    X.rng = np.random.default_rng(SEED)
    df = X.simulate()
    
    ax = axes[0]
    if integrated:
        df.cumsum().plot(kind='line', ax=ax)
    else:
        df.plot(kind='line', ax=ax)

    if X.q > 0:
        ax.set_xlabel('time\n$\mu$={:.1f}, a={}'.format(X.mu, X.b))
    else:
        ax.set_xlabel('time\n$\mu$={:.1f}'.format(X.mu))
    ax.set_ylabel('spot')
    ax.legend(loc='upper right')
    ax.set_title('MA({:.0f})'.format(X.q))
    
    ax = axes[1]
    plot_acf(df, ax=ax)
    
plt.tight_layout()
plt.show()

## Autoregressive Moving-average model ARMA(p, q)

$$X_t = \mu + Z_t + \sum_{i=1}^p a_{i} X_{t-i} + \sum_{j=1}^q b_{j} Z_{t-j}\,.$$

In [ ]:
X = ARMA(x0=[0.0, 0.0], mu=0.0, T=T, vol=1.0)

for i in range(5):
    fig, axess = plt.subplots(nrows=5, ncols=2)
    for j in range(5):
        axes = axess[j]
        a = a_[i]
        b = b_[j]
        X.a = a
        X.b = b
        X.rng = np.random.default_rng(SEED)
        df = X.simulate()

        ax = axes[0]    
        if integrated:
            df.cumsum().plot(kind='line', ax=ax)
        else:
            df.plot(kind='line', ax=ax)

        if X.p > 0 and X.q > 0:
            ax.set_xlabel('time\n$\mu$={:.1f}, a={}, b={}'.format(X.mu, X.a, X.b))
        elif X.p > 0 and X.q == 0:
            ax.set_xlabel('time\n$\mu$={:.1f}, a={}'.format(X.mu, X.a))
        elif X.p == 0 and X.q > 0:
            ax.set_xlabel('time\n$\mu$={:.1f}, b={}'.format(X.mu, X.b))
        else:
            ax.set_xlabel('time\n$\mu$={:.1f}'.format(X.mu))

        ax.set_ylabel('spot')
        ax.legend(loc='upper right')
        ax.set_title('ARMA({:.0f}, {:.0f})'.format(X.p, X.q))

        ax = axes[1]
        plot_acf(df, ax=ax)

    plt.tight_layout()
    plt.show()

## Autoregressive conditional heteroskedasticity ARCH(p)
\begin{align}
&X_t = \sigma_t Z_t\,,\\
&\sigma^2_t = v + \sum_{i=1}^p a_i X^2_{t-i}\,.
\end{align}

In [ ]:
fig, axess = plt.subplots(nrows=4, ncols=2)

a_ = [
    [],
    [0.9],
    [2.1],
    [0.3, 1.0],
]

X = ARCH(x0=[0.0, 0.0], sigma0=[1.0, 1.0], v=1.0, T=T, vol=1.0)

for i, axes in enumerate(axess):
    a = a_[i]
    X.a = a
    X.rng = np.random.default_rng(SEED)
    df = X.simulate()
    
    ax = axes[0]
    if integrated:
        df['spot'].cumsum().plot(kind='line', ax=ax)
        df['vol'].plot(kind='line', ax=ax)
    else:
        df.plot(kind='line', ax=ax)

    
    if X.p > 0:
        ax.set_xlabel('time\n$v$={:.1f}, a={}'.format(X.v, X.a))
    else:
        ax.set_xlabel('time\n$v$={:.1f}'.format(X.v))
    ax.set_ylabel('spot')
    ax.legend(loc='upper right')
    ax.set_title('ARCH({:.0f})'.format(X.p))
    
    ax = axes[1]
    plot_acf(df['vol'], ax=ax, title='Autocorrelation of volatility')
    
plt.tight_layout()
plt.show()

## Generalized autoregressive conditional heteroskedasticity GARCH(p, q)
\begin{align}
&X_t = \sigma_t Z_t\,,\\
&\sigma^2_t = v + \sum_{i=1}^p a_i X^2_{t-i} + \sum_{j=1}^q b_j \sigma^2_{t-j}\,.
\end{align}

In [ ]:
X = GARCH(x0=[0.0, 0.0], sigma0=[1.0, 1.0], v=1.0, T=T, vol=1.0)

for i in range(4):
    fig, axess = plt.subplots(nrows=4, ncols=2)
    for j in range(4):
        axes = axess[j]
        a = a_[i]
        b = a_[i]
        X.a = a
        X.b = b
        X.rng = np.random.default_rng(SEED)
        df = X.simulate()

        ax = axes[0]
        if integrated:
            df['spot'].cumsum().plot(kind='line', ax=ax)
            df['vol'].plot(kind='line', ax=ax)
        else:
            df.plot(kind='line', ax=ax)

        if X.p > 0 and X.q > 0:
            ax.set_xlabel('time\n$v$={:.1f}, a={}, b={}'.format(X.v, X.a, X.b))
        elif X.p > 0 and X.q == 0:
            ax.set_xlabel('time\n$v$={:.1f}, a={}'.format(X.v, X.a))
        elif X.p == 0 and X.q > 0:
            ax.set_xlabel('time\n$v$={:.1f}, b={}'.format(X.v, X.b))
        else:
            ax.set_xlabel('time\n$v$={:.1f}'.format(X.v))

        ax.set_ylabel('spot')
        ax.legend(loc='upper right')
        ax.set_title('GARCH({:.0f}, {:.0f})'.format(X.p, X.q))

        ax = axes[1]
        plot_acf(df['vol'], ax=ax, title='Autocorrelation of volatility')

    plt.tight_layout()
    plt.show()